# Demo 17 - Naive Bayes and Logistic Regressions

In [ ]:
import nltk
import sklearn
import pandas as pd
import numpy as np

## Multiplying Probabilities

In [ ]:
np.random.rand() * np.random.rand() * np.random.rand()

In [ ]:
rand_number = np.random.rand() 
for i in range(10):
    rand_number *= np.random.rand() 
rand_number

In [ ]:
rand_number = np.random.rand() 
for i in range(100):
    rand_number *= np.random.rand() 
rand_number

In [ ]:
rand_number = np.random.rand() 
for i in range(500):
    rand_number *= np.random.rand() 
rand_number

In [ ]:
rand_number = np.random.rand() 
for i in range(600):
    rand_number *= np.random.rand() 
rand_number

In [ ]:
rand_number = np.random.rand() 
for i in range(1000):
    rand_number *= np.random.rand() 
rand_number

(back to slides)

#### Logs to the rescue

In [ ]:
rand_number = np.log(np.random.rand())
for i in range(1000):
    rand_number += np.log(np.random.rand()) 
rand_number

## Sklearn

### Movie Reviews

In [ ]:
moview_reviews = nltk.corpus.movie_reviews

In [ ]:
review_files = [(file_id, file_id.startswith("pos")) for file_id in moview_reviews.fileids()]

len(neg_review), len(pos_review)

In [ ]:
df = pd.DataFrame(review_files)
df

In [ ]:
df = df.rename(columns={0: "file_name", 1: "gold-label"})
df

In [ ]:
def read_mov_review(f_name):
    return moview_reviews.open(f_name).read()

df['review_text'] = df['file_name'].apply(read_mov_review)

In [ ]:
df['review_text']

#### Make train/dev/test split

In [ ]:
df = df.sample(df.shape[0])
df.head(5)

In [ ]:
df.shape[0]

Let's make 80:10:10 split of our data

In [ ]:
train_max_idx = int(df.shape[0] * .8)
dev_max_idx = int((df.shape[0] * .1) + train_max_idx)


train_max_idx, dev_max_idx

In [ ]:
train_df = df.iloc[:train_max_idx]
dev_df = df.iloc[train_max_idx:dev_max_idx]
test_df = df.iloc[dev_max_idx:]

train_df.shape, dev_df.shape, test_df.shape

We will train models on `train_df`, change different hyper-parameters based on `dev_df`, and then evaluate the model on `test_df` once we are done changing our different hyperparameters.

### Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
nb_model = MultinomialNB()
nb_model

In [ ]:
nb_model.fit(train_df['review_text'], train_df['gold-label'])

**Question:** What happened?

Solution: We need to convert the text to numbers, actually features

(back to slides)

#### Feature Engineering

We need to convert each document into features. 

**Question:** What should our features be?

*Hint*: What's the most basic approach we've taken so far in this class?

<details>
<summary>Solution</summary>
    Bag of words
</details>

In [ ]:
# skip next bunch of empty cells, answer below

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=4)

In [ ]:
vectorizer.fit_transform(train_df['review_text'])

**Question:** What do these numbers that were printed out mean?

<details>
<summary>Solution</summary>
Number of examples, size of vocabulary
</details>

In [ ]:
X = vectorizer.fit_transform(train_df['review_text'])
X

Let's look at the contenxtual help to print out what X is.

**Question:** What do we think *(0, 8284) <tab>  2* in *X* means?
    
<details>
<summary>Solution</summary>
    The value of the 8284 feature for the 0-th example. In our setting this means how many times that word appeared in the first document

        
</details>

In [ ]:
vectorizer.get_feature_names()[8284]

In [ ]:
train_df['review_text'].iloc[0].count('notoriety')

#### Train the model

In [ ]:
nb_model.fit(X, train_df['gold-label'])

#### How well does the model perform on the training set?

In [ ]:
nb_model.score(X, train_df['gold-label'])

If the model just guessed True or False:

In [ ]:
train_df['gold-label'].value_counts(normalize=True)

#### Evaluate the model on dev

In [ ]:
nb_model.predict(dev_df['review_text'])

In [ ]:
X_dev = vectorizer.transform(dev_df['review_text'])
X_dev

In [ ]:
nb_model.predict(X_dev)

**Let's store the predictions**

In [ ]:
dev_df = dev_df.assign(nb_predictions = nb_model.predict(X_dev))
dev_df.keys()

**Let's compute accuracy**

In [ ]:
dev_df['nb_predictions'] == dev_df['gold-label']

In [ ]:
sum(dev_df['nb_predictions'] == dev_df['gold-label']) / dev_df.shape[0]

#### Interpreting the model

In computational text analysis, we don't necessarily care just about building a classifier that does well. We want to use the classifier to gain insight about our text.

**Question:** From the Naive Bayes equation, what do you think can give us the most insight about our text?

In [ ]:
nb_model.feature_log_prob_.shape

In [ ]:
nb_model.feature_log_prob_[0]

In [ ]:
nb_model.feature_log_prob_[0].argmax()

In [ ]:
vectorizer.get_feature_names()[nb_model.feature_log_prob_[0].argmax()]

In [ ]:
nb_model.feature_log_prob_[0].max(), nb_model.feature_log_prob_[0][nb_model.feature_log_prob_[0].argmax()]

In [ ]:
# code from https://stackoverflow.com/questions/50526898/how-to-get-feature-importance-in-naive-bayes

neg_class_prob_sorted = nb_model.feature_log_prob_[0, :].argsort()[::-1]
pos_class_prob_sorted = nb_model.feature_log_prob_[1, :].argsort()[::-1]

print(np.take(vectorizer.get_feature_names(), neg_class_prob_sorted[:10]))
print(np.take(vectorizer.get_feature_names(), pos_class_prob_sorted[:10]))

The above is telling us what is the probability of a word given a label.

In [ ]:
neg_class_prob_sorted = nb_model.feature_log_prob_[0, :].argsort()[::-1]
pos_class_prob_sorted = nb_model.feature_log_prob_[1, :].argsort()[::-1]

top_100_negative = set(np.take(vectorizer.get_feature_names(), neg_class_prob_sorted[:200]))
top_100_positive = set(np.take(vectorizer.get_feature_names(), pos_class_prob_sorted[:200]))

In [ ]:
top_100_negative - top_100_positive

In [ ]:
top_100_positive - top_100_negative

#### Fine-tuning hyper-parameters

In [ ]:
nb_model.alpha = 5
nb_model.alpha

In [ ]:
nb_model = MultinomialNB(alpha=5)

In [ ]:
X_train

In [ ]:
nb_model.fit(X, train_df['gold-label'])

In [ ]:
nb_model.score(X, train_df['gold-label'])

In [ ]:
nb_model.predict(X_dev)

In [ ]:
nb_model.score(X_dev, dev_df['gold-label'])

**How does this compare to the result on dev before?**

This was an example of hyper-parameter tuning

(back to slides)
### Logistic Regression

In [ ]:
lr_model = sklearn.linear_model.LogisticRegression(max_iter = 1e4)
lr_model

In [ ]:
lr_model.fit()

In [ ]:
lr_model.fit(df['review_text'], df['gold-label'])

**Question:** What happened?


<details>
<summary>Solution</summary>
We need to convert the text to features
</details>


In [ ]:
df['review_text']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=4)

In [ ]:
X = vectorizer.fit_transform(df['review_text'])
X

**Question:** What is 2000 and what is 15452?

In [ ]:
lr_model.fit(X, df['gold-label'])

In [ ]:
df['prediction'] = lr_model.predict(X)
df['prediction']

In [ ]:
sum(df['prediction'] == df['gold-label'])

#### Finding coefficients

In [ ]:
lr_model.coef_

In [ ]:
lr_model.coef_.shape

In [ ]:
len(vectorizer.get_feature_names())

In [ ]:
lr_model.coef_.argmax()

In [ ]:
lr_model.coef_[0][lr_model.coef_.argmax()]

In [ ]:
vectorizer.get_feature_names()[lr_model.coef_.argmax()]

In [ ]:
lr_model.coef_.argmin()

In [ ]:
lr_model.coef_[0][lr_model.coef_.argmin()]

In [ ]:
vectorizer.get_feature_names()[lr_model.coef_.argmin()]

#### Prediction

In [ ]:
df

In [ ]:
df.sample(df.shape[0])

In [ ]:
df = df.sample(df.shape[0])
train_df = df.head(1800)
test_df = df.tail(200)

train_df.shape, test_df.shape

In [ ]:
vectorizer = CountVectorizer(min_df=4)
vectorizer.fit(train_df['review_text'])

In [ ]:
X_train = vectorizer.transform(train_df)
X_train

In [ ]:
lr_model = sklearn.linear_model.LogisticRegression(max_iter = 1e4)
lr_model

In [ ]:
lr_model.fit(X_train, train_df['gold-label'])

#### n-grams

Look at ngram_range in https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

(back to slides)
### More classifiers

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=(5, 2), random_state=1)
clf = SVC(gamma=2, C=1)
clf = DecisionTreeClassifier(max_depth=5)